In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision import transforms, utils

from torchvision import models

from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import balanced_accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

import time
import copy
import pickle

In [2]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA is supported


In [3]:
load_transform = transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
                                    # same normalize as EfficientNet paper
# cifar-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=load_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=load_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

In [4]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b1')

Loaded pretrained weights for efficientnet-b1


In [5]:
# change fc output class numbers
model._fc.out_features = 10

# freezing convolutional weights
#for param in model._blocks.parameters():
#    param.requires_grad = False
# not freeze convoluntional layers for this time

In [10]:
# test and return accuracy
def testing(model, testloader, criterion, computing_device):
    # make sure model is in evaluation mode
    model.eval()
    val_loss = 0.0
    val_correct = 0
    test_batch_loss = []
    for data in testloader:
        data, target = data[0].to(computing_device), data[1].to(computing_device)
        output = model(data)
        loss = criterion(output, target)
        
        #test_batch_loss.append(loss)
        val_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_correct += (preds == target).sum().item()
    
    val_loss = val_loss/len(testloader.dataset)
    val_accuracy = 100.0 * val_correct/len(testloader.dataset)
    
    return val_loss, val_accuracy

def training(model, optimizer, trainloader, criterion, computing_device):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for data in trainloader:
        data, target = data[0].to(computing_device), data[1].to(computing_device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_loss/len(trainloader.dataset)
    train_accuracy = 100.0 * train_correct/len(trainloader.dataset)
    
    return train_loss, train_accuracy

In [11]:
n_epoch = 30
train_loss = []
train_acc = []
test_loss = []
test_acc = []

# define optimizer and loss
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# bring model to computing device
model = model.to(computing_device)
best_model_weights = copy.deepcopy(model.state_dict()) 

# start training with early stopping
for epoch in range(n_epoch):
    loss,acc = training(model, optimizer, trainloader, criterion, computing_device)
    train_loss.append(loss)
    train_acc.append(acc)
    print("Epoch" + str(epoch) + " : train loss " + str(loss) + ", train accuracy " + str(acc))
    
    loss,acc = testing(model, testloader, criterion, computing_device)
    test_loss.append(loss)
    test_acc.append(acc)
    print("Epoch" + str(epoch) + " : test loss " + str(loss) + ", test accuracy " + str(acc))

    
    ##################early stop####################
    # early stop if validation loss increased
    if(epoch > 5):
        before = test_loss[-2]
        after = test_loss[-1]
        if (before < after and test_acc[-2] > test_acc[-1]):
            break
        else:
            best_model_weights = copy.deepcopy(model.state_dict())
            
model.load_state_dict(best_model_weights)   

Epoch0 : train loss 0.029098667112886906, train accuracy 77.982
Epoch0 : test loss 0.007227938933670521, test accuracy 92.83
Epoch1 : train loss 0.007433852560818195, train accuracy 92.156
Epoch1 : test loss 0.005735882423818111, test accuracy 94.24
Epoch2 : train loss 0.005221331487298012, train accuracy 94.4
Epoch2 : test loss 0.004695287980139256, test accuracy 95.41
Epoch3 : train loss 0.0038787417989969253, train accuracy 95.814
Epoch3 : test loss 0.003956038857996464, test accuracy 96.01
Epoch4 : train loss 0.003081741962134838, train accuracy 96.728
Epoch4 : test loss 0.0036610833182930944, test accuracy 96.4
Epoch5 : train loss 0.002483963512778282, train accuracy 97.34
Epoch5 : test loss 0.003542520788311958, test accuracy 96.42
Epoch6 : train loss 0.002105522178411484, train accuracy 97.746
Epoch6 : test loss 0.003467254935204983, test accuracy 96.58
Epoch7 : train loss 0.0017796184101700782, train accuracy 98.066
Epoch7 : test loss 0.0033724164709448812, test accuracy 96.82


<All keys matched successfully>

In [14]:
# save model weights
torch.save(model.state_dict(), 'model/efficient_net_b1_cifar10.pth')

In [15]:
# save results
np.save('result/train_loss_b1_basseline.npy', train_loss)
np.save('result/train_acc_b1_basseline.npy', train_acc)
np.save('result/test_loss_b1_basseline.npy', test_loss)
np.save('result/test_acc_b1_basseline.npy', test_acc)

### adv testing